In [121]:
import logging
import numpy as np
import pandas as pd
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [88]:
data_train = pd.read_csv("/Users/revan/Downloads/congressional_tweet_training_data.csv")
data_test = pd.read_csv("/Users/revan/Downloads/congressional_tweet_test_data.csv")

In [89]:
#df1.rename(
 #   columns={"party":"party_id"},inplace=True)

In [90]:
#frames = [df1, df2]

#data = pd.concat(frames)

In [91]:
df_train = data_train[["full_text","party_id"]]
df_test = data_test[["full_text","party"]]
df_test.head()

,full_text,party
0,b'#TaxReform improved the playing field for Am...,D
1,"b'This #NativeWomensEqualPay Day, we recommit ...",D
2,"b""\xe2\x80\x9cI became convinced that our gene...",D
3,"b'During #NationalAdoptionMonth, we honor the ...",D
4,b'Happy #AirborneDay to our @USArmy paratroope...,D


In [92]:
df_train['full_text'] = df_train['full_text'].str[1:]
df_test['full_text'] = df_test['full_text'].str[1:]

/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/477920869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['full_text'] = df_train['full_text'].str[1:]
/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/477920869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['full_text'] = df_test['full_text'].str[1:]


In [93]:
df_test.head()

,full_text,party
0,'#TaxReform improved the playing field for Ame...,D
1,"'This #NativeWomensEqualPay Day, we recommit t...",D
2,"""\xe2\x80\x9cI became convinced that our gener...",D
3,"'During #NationalAdoptionMonth, we honor the a...",D
4,'Happy #AirborneDay to our @USArmy paratrooper...,D


In [94]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [96]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df_train['clean_msg']= df_train['full_text'].apply(lambda x:remove_punctuation(x))
df_test['clean_msg']= df_test['full_text'].apply(lambda x:remove_punctuation(x))

/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/899601270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['clean_msg']= df_train['full_text'].apply(lambda x:remove_punctuation(x))
/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/899601270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['clean_msg']= df_test['full_text'].apply(lambda x:remove_punctuation(x))


In [97]:
df_test.head()

,full_text,party,clean_msg
0,'#TaxReform improved the playing field for Ame...,D,TaxReform improved the playing field for Ameri...
1,"'This #NativeWomensEqualPay Day, we recommit t...",D,This NativeWomensEqualPay Day we recommit to p...
2,"""\xe2\x80\x9cI became convinced that our gener...",D,xe2x80x9cI became convinced that our generatio...
3,"'During #NationalAdoptionMonth, we honor the a...",D,During NationalAdoptionMonth we honor the adop...
4,'Happy #AirborneDay to our @USArmy paratrooper...,D,Happy AirborneDay to our USArmy paratroopers a...


In [98]:
df_train['msg_lower']= df_train['clean_msg'].apply(lambda x: x.lower())
df_test['msg_lower']= df_test['clean_msg'].apply(lambda x: x.lower())

/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/861870578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['msg_lower']= df_train['clean_msg'].apply(lambda x: x.lower())
/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/861870578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['msg_lower']= df_test['clean_msg'].apply(lambda x: x.lower())


In [99]:
import re
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens
#applying function to the column
df_train['msg_tokenied']= df_train['msg_lower'].apply(lambda x: tokenization(x))
df_test['msg_tokenied']= df_test['msg_lower'].apply(lambda x: tokenization(x))

/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/3181606943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['msg_tokenied']= df_train['msg_lower'].apply(lambda x: tokenization(x))
/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/3181606943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['msg_tokenied']= df_test['msg_lower'].apply(lambda x: tokenization(x))


In [100]:
df_test.head()

,full_text,party,clean_msg,msg_lower,msg_tokenied
0,'#TaxReform improved the playing field for Ame...,D,TaxReform improved the playing field for Ameri...,taxreform improved the playing field for ameri...,[taxreform improved the playing field for amer...
1,"'This #NativeWomensEqualPay Day, we recommit t...",D,This NativeWomensEqualPay Day we recommit to p...,this nativewomensequalpay day we recommit to p...,[this nativewomensequalpay day we recommit to ...
2,"""\xe2\x80\x9cI became convinced that our gener...",D,xe2x80x9cI became convinced that our generatio...,xe2x80x9ci became convinced that our generatio...,[xe2x80x9ci became convinced that our generati...
3,"'During #NationalAdoptionMonth, we honor the a...",D,During NationalAdoptionMonth we honor the adop...,during nationaladoptionmonth we honor the adop...,[during nationaladoptionmonth we honor the ado...
4,'Happy #AirborneDay to our @USArmy paratrooper...,D,Happy AirborneDay to our USArmy paratroopers a...,happy airborneday to our usarmy paratroopers a...,[happy airborneday to our usarmy paratroopers ...


In [101]:
import nltk
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

In [102]:
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [103]:
df_train['no_stopwords']= df_train['msg_tokenied'].apply(lambda x:remove_stopwords(x))
df_test['no_stopwords']= df_test['msg_tokenied'].apply(lambda x:remove_stopwords(x))

/var/folders/79/l_dmwqfj6v94294gmn9b0rs80000gn/T/ipykernel_37289/924903993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['no_stopwords']= df_test['msg_tokenied'].apply(lambda x:remove_stopwords(x))


In [104]:
df_test.head()

,full_text,party,clean_msg,msg_lower,msg_tokenied,no_stopwords
0,'#TaxReform improved the playing field for Ame...,D,TaxReform improved the playing field for Ameri...,taxreform improved the playing field for ameri...,[taxreform improved the playing field for amer...,[taxreform improved the playing field for amer...
1,"'This #NativeWomensEqualPay Day, we recommit t...",D,This NativeWomensEqualPay Day we recommit to p...,this nativewomensequalpay day we recommit to p...,[this nativewomensequalpay day we recommit to ...,[this nativewomensequalpay day we recommit to ...
2,"""\xe2\x80\x9cI became convinced that our gener...",D,xe2x80x9cI became convinced that our generatio...,xe2x80x9ci became convinced that our generatio...,[xe2x80x9ci became convinced that our generati...,[xe2x80x9ci became convinced that our generati...
3,"'During #NationalAdoptionMonth, we honor the a...",D,During NationalAdoptionMonth we honor the adop...,during nationaladoptionmonth we honor the adop...,[during nationaladoptionmonth we honor the ado...,[during nationaladoptionmonth we honor the ado...
4,'Happy #AirborneDay to our @USArmy paratrooper...,D,Happy AirborneDay to our USArmy paratroopers a...,happy airborneday to our usarmy paratroopers a...,[happy airborneday to our usarmy paratroopers ...,[happy airborneday to our usarmy paratroopers ...


In [105]:
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [106]:
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
df_train['msg_stemmed']=df_train['no_stopwords'].apply(lambda x: stemming(x))
df_test['msg_stemmed']=df_test['no_stopwords'].apply(lambda x: stemming(x))

In [107]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatizationfid
wordnet_lemmatizer = WordNetLemmatizer()

In [108]:
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df_train['msg_lemmatized']=df_train['msg_stemmed'].apply(lambda x:lemmatizer(x))
df_test['msg_lemmatized']=df_test['msg_stemmed'].apply(lambda x:lemmatizer(x))

In [109]:
df_test.head()

,full_text,party,clean_msg,msg_lower,msg_tokenied,no_stopwords,msg_stemmed,msg_lemmatized
0,'#TaxReform improved the playing field for Ame...,D,TaxReform improved the playing field for Ameri...,taxreform improved the playing field for ameri...,[taxreform improved the playing field for amer...,[taxreform improved the playing field for amer...,[taxreform improved the playing field for amer...,[taxreform improved the playing field for amer...
1,"'This #NativeWomensEqualPay Day, we recommit t...",D,This NativeWomensEqualPay Day we recommit to p...,this nativewomensequalpay day we recommit to p...,[this nativewomensequalpay day we recommit to ...,[this nativewomensequalpay day we recommit to ...,[this nativewomensequalpay day we recommit to ...,[this nativewomensequalpay day we recommit to ...
2,"""\xe2\x80\x9cI became convinced that our gener...",D,xe2x80x9cI became convinced that our generatio...,xe2x80x9ci became convinced that our generatio...,[xe2x80x9ci became convinced that our generati...,[xe2x80x9ci became convinced that our generati...,[xe2x80x9ci became convinced that our generati...,[xe2x80x9ci became convinced that our generati...
3,"'During #NationalAdoptionMonth, we honor the a...",D,During NationalAdoptionMonth we honor the adop...,during nationaladoptionmonth we honor the adop...,[during nationaladoptionmonth we honor the ado...,[during nationaladoptionmonth we honor the ado...,[during nationaladoptionmonth we honor the ado...,[during nationaladoptionmonth we honor the ado...
4,'Happy #AirborneDay to our @USArmy paratrooper...,D,Happy AirborneDay to our USArmy paratroopers a...,happy airborneday to our usarmy paratroopers a...,[happy airborneday to our usarmy paratroopers ...,[happy airborneday to our usarmy paratroopers ...,[happy airborneday to our usarmy paratroopers ...,[happy airborneday to our usarmy paratroopers ...


In [110]:
df_test['msg_lemmatized'][2]

['xe2x80x9ci became convinced that our generation and our silence makes us complicit in this environment of rampant sexual assault and sexual harassment in the workplace so wexe2x80x99re here to say no morexe2x80x9d  metoo shatteringthesilence more in thehill httpstcod6yvmi919f']

In [111]:
def no_na(text):
    result = [x for x in text if x != 'NaN']
    return result
df_train['full']=df_train['msg_lemmatized'].apply(lambda x:no_na(x))
df_test['full']=df_test['msg_lemmatized'].apply(lambda x:no_na(x))

In [112]:
df_train['text_clean'] = df_train['full'].apply(lambda x: ','.join(map(str, x)))
df_test['text_clean'] = df_test['full'].apply(lambda x: ','.join(map(str, x)))

In [113]:
df_test["text_clean"].head()

0    taxreform improved the playing field for ameri...
1    this nativewomensequalpay day we recommit to p...
2    xe2x80x9ci became convinced that our generatio...
3    during nationaladoptionmonth we honor the adop...
4    happy airborneday to our usarmy paratroopers a...
Name: text_clean, dtype: object

In [114]:
type(df_train["text_clean"][0])

str

In [115]:
#df = pd.concat(frames)

In [118]:
data_train["clean_text"] = df_train["text_clean"]
data_test["clean_text"] = df_test["text_clean"]

In [124]:
def linearscv(df):    
    #print(df["clean_text"])
    print("starting linearscv")
    x = df['clean_text'].tolist()
    tfid = TfidfVectorizer(ngram_range=(1,2), max_features=10000, lowercase= True)
    tf = tfid.fit_transform(x)
    print('No. of feature_words: ', tfid.get_feature_names())
    return (tf)

def accuracy(df_train, df_test):
    y_train = df_train['party_id'].map({'D': 0, 'R': 1}).tolist()
    y_test = df_test['party'].map({'D': 0, 'R': 1}).tolist()
    x_train = linearscv(df_train)
    x_test = linearscv(df_test)
    #print(x_train.shape, x_test.shape)
    print("starting regression")
    clf1 = LogisticRegression(C=2, random_state = 265, max_iter=3000, n_jobs = -1).fit(x_train, y_train)
    clf2 = LinearSVC().fit(x_train, y_train)
    clf3 = LogisticRegressionCV(cv = 5, random_state = 265, max_iter=3000, n_jobs = -1).fit(x_train, y_train)
    score1 = clf1.score(x_test, y_test)
    score2 = clf2.score(x_test, y_test)
    score3 = clf3.score(x_test, y_test)
    y_predict = clf1.predict(x_test) 
    print(y_predict)
    results = [] 
    results.append(["LR" , score1])
    results.append(["SVC" , score2])
    results.append(["LRCV" , score3])
    print(pd.DataFrame(results, columns=['label', 'accuracy']))
    return  y_predict    

In [ ]:
accuracy(data_train, data_test)

starting linearscv


/Users/revan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


No. of feature_words:  10000
starting linearscv
No. of feature_words:  10000
starting regression
[1 0 0 ... 1 1 0]
  label  accuracy
0    LR  0.482570
1   SVC  0.461098
2  LRCV  0.483525


In [32]:
#df.to_csv('cleaned_test.csv')
#df.to_csv('cleaned_train.csv')